In [10]:
import pandas as pd
import matplotlib.pyplot as plt
# import keras
from sklearn.utils import shuffle
%matplotlib inline
import numpy as np
data = pd.read_csv(r'./SMILES_entry.csv',sep=',',usecols=[1])

In [11]:
print(data.head())

                               SMILES
0              OOC1(OO)CCC(OO)(OO)CC1
1         OOC(OO)C1=CC=C(C(OO)OO)C=C1
2              ON1N=NN=C1NNC2=NN=NN2O
3  ON1N=NN=C1CN2N=NC([N+]([O-])=O)=N2
4      ON1N=NN=C1C2=NON=C2N=[N+]=[N-]


In [12]:
charset = set("".join(list(data.SMILES)))
print(len(charset))
char_to_int = dict((c,i) for i,c in enumerate(charset))
int_to_char = dict((i,c) for i,c in enumerate(charset))
embed = max([len(smile) for smile in data.SMILES]) + 5
print (str(charset))
print(len(charset), embed)

29
{'@', '8', '#', '0', 'H', '.', '\\', '(', '2', ')', '-', 'N', 'C', ' ', '1', ']', 'O', '4', '7', '9', '%', '=', '3', '/', '6', '+', '[', 'S', '5'}
29 370


In [12]:
# char_to_int 

In [13]:
def vectorize(smiles):
        one_hot =  np.zeros((smiles.shape[0], embed , len(charset)),dtype=np.int8)
        for i,smile in enumerate(smiles):
            #encode the startchar
            one_hot[i,0,char_to_int["!"]] = 1
            #encode the rest of the chars
            for j,c in enumerate(smile):
                one_hot[i,j+1,char_to_int[c]] = 1
            #Encode endchar
            one_hot[i,len(smile)+1:,char_to_int["E"]] = 1
        #Return two, one for input and the other for output
        return one_hot[:,0:-1,:], one_hot[:,1:,:]

In [14]:
X_train, _ = vectorize(X_train)
X_test, _ = vectorize(X_test)

In [90]:
X_train.dtype

dtype('int8')

In [17]:
# X_train[8]

In [18]:
mol_str_train=[]
mol_str_test=[]
for x in range(3945):
 
    mol_str_train.append("".join([int_to_char[idx] for idx in np.argmax(X_train[x,:,:], axis=1)]))
    
    
for x in range(1315):
    mol_str_test.append("".join([int_to_char[idx] for idx in np.argmax(X_test[x,:,:], axis=1)]))

# mol_str_train_data=pd.DataFrame(data=mol_str_train)
# mol_str_test_data=pd.DataFrame(data=mol_str_test)
# mol_str_train_data.to_csv("mol_str_train.csv")
# mol_str_test_data.to_csv("mol_str_test.csv")

In [21]:
for x in range(3945):
    a = np.argmax(X_train[x,:,:], axis=1)
print(a.max())
for x in range(1315):
    b = np.argmax(X_train[x,:,:], axis=1)
print(b.max())    

29
31


In [22]:
vocab_size=len(charset)
vocab_size

34

In [95]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.layers.embeddings import Embedding

In [96]:
print("Keras: %s"%keras.__version__)

Keras: 2.3.1


In [104]:
model = Sequential()
model.add(Embedding(vocab_size, 68, input_length=embed-1)) # input_dim, output_dim
model.add(keras.layers.Conv1D(50,10,activation='relu'))
# （批）规范化BatchNormalization
# 该层在每个batch上将前一层的激活值重新规范化，即使得其输出数据的均值接近0，其标准差接近1
model.add(BatchNormalization())
model.add(keras.layers.Conv1D(50,5,activation='relu'))
model.add(keras.layers.Conv1D(50,3,activation='relu'))
model.add(Flatten())
# model.add(Dense(60, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='linear'))

In [82]:
def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [83]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

In [84]:
# mol_str_train = np.asarray(mol_str_train)
# mol_str_test = np.asarray(mol_str_test)

In [107]:
optimizer = keras.optimizers.Adam(lr=0.00025)
lr_metric = get_lr_metric(optimizer)
model.compile(loss="mse", optimizer=optimizer, metrics=[coeff_determination, lr_metric])

In [108]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 213, 68)           2312      
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 204, 50)           34050     
_________________________________________________________________
batch_normalization_18 (Batc (None, 204, 50)           200       
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 200, 50)           12550     
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 198, 50)           7550      
_________________________________________________________________
flatten_18 (Flatten)         (None, 9900)              0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 9900)            

In [111]:
from keras.callbacks import ReduceLROnPlateau
# 定义学习率之后，经过一定epoch迭代之后，模型效果不再提升，该学习率可能已经不再适应该模型。需要在训练过程中缩小学习率，进而提升模型。
# 使用keras中的回调函数ReduceLROnPlateau

# monitor：监测的值，可以是accuracy，val_loss,val_accuracy
# factor：缩放学习率的值，学习率将以lr = lr*factor的形式被减少
# patience：当patience个epoch过去而模型性能不提升时，学习率减少的动作会被触发
# mode：‘auto’，‘min’，‘max’之一 默认‘auto’就行
# epsilon：阈值，用来确定是否进入检测值的“平原区”
# cooldown：学习率减少后，会经过cooldown个epoch才重新进行正常操作
# min_lr：学习率最小值，能缩小到的下限
# ————————————————
# monitor="val_loss",
#     factor=0.1,
#     patience=10,
#     verbose=0,
#     mode="auto",
#     min_delta=0.0001,
#     cooldown=0,
#     min_lr=0,
# 原文链接：https://blog.csdn.net/weixin_44048809/article/details/105711356
    
callbacks_list = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-15, verbose=0, mode='auto',cooldown=0),
    ModelCheckpoint(filepath="weights.best.hdf5", monitor='val_loss', save_best_only=True, verbose=1, mode='auto')   
]

history =model.fit(x=np.argmax(X_train, axis=2), y=Y_train,
                              batch_size=128,
                              epochs=50,
                              validation_data=(np.argmax(X_test, axis=2),Y_test),
                              callbacks=callbacks_list)

Train on 3945 samples, validate on 1315 samples
Epoch 1/50
3945/3945 [==============================] - 11s 3ms/step - loss: 26232040.7483 - coeff_determination: 0.2572 - lr: 2.5000e-04 - val_loss: 138687513.7764 - val_coeff_determination: -3.1455 - val_lr: 2.5000e-04

Epoch 00001: val_loss improved from inf to 138687513.77643, saving model to weights.best.hdf5
Epoch 2/50
3945/3945 [==============================] - 11s 3ms/step - loss: 25852909.7850 - coeff_determination: 0.2573 - lr: 2.5000e-04 - val_loss: 162890142.2966 - val_coeff_determination: -3.8752 - val_lr: 2.5000e-04

Epoch 00002: val_loss did not improve from 138687513.77643
Epoch 3/50
3945/3945 [==============================] - 11s 3ms/step - loss: 25791548.1379 - coeff_determination: 0.2667 - lr: 2.5000e-04 - val_loss: 62598946.9293 - val_coeff_determination: -0.8639 - val_lr: 2.5000e-04

Epoch 00003: val_loss improved from 138687513.77643 to 62598946.92928, saving model to weights.best.hdf5
Epoch 4/50
3945/3945 [=======


Epoch 00028: val_loss did not improve from 23539866.21749
Epoch 29/50
3945/3945 [==============================] - 11s 3ms/step - loss: 23404857.6862 - coeff_determination: 0.3382 - lr: 2.5000e-06 - val_loss: 23560129.8814 - val_coeff_determination: 0.3011 - val_lr: 2.5000e-06

Epoch 00029: val_loss did not improve from 23539866.21749
Epoch 30/50
3945/3945 [==============================] - 11s 3ms/step - loss: 23325698.4796 - coeff_determination: 0.3384 - lr: 2.5000e-06 - val_loss: 23543024.2692 - val_coeff_determination: 0.3014 - val_lr: 2.5000e-06

Epoch 00030: val_loss did not improve from 23539866.21749
Epoch 31/50
3945/3945 [==============================] - 12s 3ms/step - loss: 23253400.1222 - coeff_determination: 0.3395 - lr: 2.5000e-06 - val_loss: 23545869.9255 - val_coeff_determination: 0.3013 - val_lr: 2.5000e-06

Epoch 00031: val_loss did not improve from 23539866.21749
Epoch 32/50
3945/3945 [==============================] - 11s 3ms/step - loss: 23272491.2943 - coeff_dete

In [ ]:
np.argmax(X_test, axis=2)

In [23]:
hist = history.history

plt.figure(figsize=(10, 8))

for label in ['val_coeff_determination','coeff_determination']:
    plt.subplot(221)
    plt.plot(hist[label], label = label)
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("coeff_determination")
    
for label in ['val_loss','loss']:
    plt.subplot(222)
    plt.plot(hist[label], label = label)
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("loss")



plt.subplot(223)
plt.plot( hist['lr'],hist['val_coeff_determination']  )
plt.legend()
plt.xlabel("lr")
plt.ylabel("val_coeff_determination")


plt.subplot(224)
plt.plot( hist['lr'],hist['val_loss']  )
plt.legend()
plt.xlabel("lr")
plt.ylabel("val_loss")

    
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                    wspace=0.35)

NameError: name 'history' is not defined